# This notebook has moved

This notebook has moved to [RI Fraud Classification Walkthrough in Google Colab](https://colab.research.google.com/github/RobustIntelligence/docs/blob/main/notebooks/demo_notebooks/RI_Fraud_Classification_Walkthrough.ipynb).
